In [24]:
print("okay")

okay


In [25]:
pip install -U langchain-community


Note: you may need to restart the kernel to use updated packages.


In [26]:
pip install --upgrade pinecone-client langchain


Note: you may need to restart the kernel to use updated packages.


In [27]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [28]:
PINECONE_API_KEY = "pcsk_72qGoG_GRkW3AESbc8HmzZYUs54xCZgXaoUpj2UJZpN8HJYm8vNfAKiWh3TkTW144WSQnu"
PINECONE_API_ENV = "aws-free"
PINECONE_CLOUD = "aws"



In [29]:
#Extracting data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [30]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [31]:
extracted_data = load_pdf("data/")

In [32]:
for doc in extracted_data:
    print(doc.page_content)  # Print the text content
    print(doc.metadata) 

The GALE
ENCYCLOPEDIA
of MEDICINE
SECOND EDITION
{'source': 'data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf', 'page': 0}
The G ALE
ENCYCLOPEDIA
of M EDICINE
SECOND EDITION
JACQUELINE L. LONGE, EDITOR
DEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR
VOLUME
C-F
2
{'source': 'data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf', 'page': 1}
STAFF
Jacqueline L. Longe,Project Editor
Deirdre S. Blanchfield, Associate Editor
Christine B. Jeryan, Managing Editor
Donna Olendorf, Senior Editor
Stacey Blachford, Associate Editor
Kate Kretschmann, Melissa C. McDade, Ryan
Thomason, Assistant Editors
Mark Springer, Technical Specialist
Andrea Lopeman, Programmer/Analyst
Barbara J. Yarrow,Manager, Imaging and Multimedia
Content
Robyn V . Young,Project Manager, Imaging and
Multimedia Content
Dean Dauphinais, Senior Editor, Imaging and
Multimedia Content
Kelly A. Quin, Editor, Imaging and Multimedia Content
Leitha Etheridge-Sims, Mary K. Grimes, Dave Oblender,
Image Catalogers
Pamela A. Reed, Imaging Coordin

In [33]:
#Creating text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [34]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 6970


Storage Calculation for Embedding Model (384 Dimensions)
Vector Size:

Each dimension in the vector is stored as a float32, which requires 4 bytes.
Vector size = 384 * 4 bytes = 1536 bytes (1.5 KB) per vector.
Storage for 14 Vectors (Chunks):

Total storage per PDF = 14 vectors * 1536 bytes = 21504 bytes (21.5 KB).
Storage Limit for Pinecone Free Tier:

Free-tier limit: 1 GB = 1,000,000 KB.
Number of PDFs you can store = 1,000,000 KB / 21.5 KB ≈ 46,511 PDFs.

In [35]:
pip install sentence-transformers

In [36]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [37]:
embeddings = download_hugging_face_embeddings()

In [38]:
embeddings


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

testing the embedding model

In [39]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [40]:
query_result

[-0.03447727486491203,
 0.03102312609553337,
 0.006734980270266533,
 0.026108933612704277,
 -0.03936205804347992,
 -0.16030246019363403,
 0.06692394614219666,
 -0.006441438104957342,
 -0.047450482845306396,
 0.014758863486349583,
 0.07087534666061401,
 0.05552757531404495,
 0.019193356856703758,
 -0.02625126577913761,
 -0.01010954286903143,
 -0.026940442621707916,
 0.022307462990283966,
 -0.02222665585577488,
 -0.14969263970851898,
 -0.017493024468421936,
 0.007676282897591591,
 0.054352231323719025,
 0.0032544038258492947,
 0.03172588348388672,
 -0.08462139964103699,
 -0.029405992478132248,
 0.051595550030469894,
 0.048124078661203384,
 -0.003314835485070944,
 -0.05827915295958519,
 0.04196925833821297,
 0.022210702300071716,
 0.1281888633966446,
 -0.022338951006531715,
 -0.011656239628791809,
 0.06292837113142014,
 -0.03287634998559952,
 -0.09122604131698608,
 -0.03117534890770912,
 0.052699536085128784,
 0.04703483358025551,
 -0.08420310169458389,
 -0.030056182295084,
 -0.0207448396

In [41]:
pip install google

Note: you may need to restart the kernel to use updated packages.


In [42]:
pip install google-api-core

In [43]:
%pip install -qU langchain-pinecone pinecone-notebooks

Note: you may need to restart the kernel to use updated packages.


In [44]:
pip install git+https://github.com/Anshit2723/medical_chatbot_project/langchain_pinecone.git@main


  Cloning https://github.com/Anshit2723/medical_chatbot_project/langchain_pinecone.git (to revision main) to c:\users\chatu\appdata\local\temp\pip-req-build-d0vyiu3b
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/Anshit2723/medical_chatbot_project/langchain_pinecone.git 'C:\Users\chatu\AppData\Local\Temp\pip-req-build-d0vyiu3b'
  remote: Not Found
  fatal: repository 'https://github.com/Anshit2723/medical_chatbot_project/langchain_pinecone.git/' not found
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/Anshit2723/medical_chatbot_project/langchain_pinecone.git 'C:\Users\chatu\AppData\Local\Temp\pip-req-build-d0vyiu3b' did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/Anshit2723/medical_chatbot_project/langchain_pinecone.git 'C:\Users\chatu\AppData\Local\Temp\pip-req-build-d0vyiu3b' did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error

In [45]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY


In [46]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [47]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)


In [48]:

docsearch

In [49]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [50]:
retrieved_docs = retriever.invoke("What is Acne?")



In [51]:
retrieved_docs

[Document(metadata={'page': 425.0, 'source': 'data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf'}, page_content='Corticosteriod—A group of synthetic hormones\nthat are used to prevent or reduce inflammation.\nToxic effects may result from rapid withdrawal after\nprolonged use or from continued use of large doses.\nPatch test—A skin test that is done to identify aller-\ngens. A suspected substance is applied to the skin.\nAfter 24–48 hours, if the area is red and swollen,\nthe test is positive for that substance. If no reaction\noccurs, another substance is applied. This is con-'),
 Document(metadata={'page': 425.0, 'source': 'data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf'}, page_content='Corticosteriod—A group of synthetic hormones\nthat are used to prevent or reduce inflammation.\nToxic effects may result from rapid withdrawal after\nprolonged use or from continued use of large doses.\nPatch test—A skin test that is done to identify aller-\ngens. A suspected substance is applied

In [52]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [53]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [54]:
pip install CTransformers

Note: you may need to restart the kernel to use updated packages.


In [55]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [56]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [58]:
while True:
    user_input = input("Input Prompt: ")
    
    if user_input.lower() == 'exit':  # Condition to exit the loop
        print("Exiting the program...")
        break
    
    result = qa({"query": user_input})  # Process the user input with the `qa` function
    print("Response:", result["result"])  # Display the response


Response: Acne is not caused by corticosteroids or the immune response, but rather it is a skin condition characterized by the presence of comedones (blackheads and whiteheads), pimples, and other inflammatory lesions on the skin.
Exiting the program...
